#Домашняя работа 5

# LightFM

Формула эмбеддинга в ALS (матричной факторизации)

$$user_{emb} = \theta_1 * i_1 + ... + \theta_n * i_n$$  
  
$$p = user_{emb} * item_{emb}^T + b_u + b_i$$

Формула в LightFM

$$user_{emb} = (\theta_1 * i_1 + ... + \theta_n * i_n) + (\alpha_1 * f_1 + ... + \alpha_k * f_k)$$  
  
$$p = user_{emb} * item_{emb}^T + b_u + b_i$$

### Зачем нужны гибридные системы?

*Холодный старт* - ситуация, в которой с товаром/юзером в матрице мало взаимодействий

Холодный страт для товаров - обычно введение нового товара (YouTube, Ozon, hh.ru)  
Холодный старт для юзера - к нам пришел новый юзер  

Холодного юзера/товар можно описать фичами --> мы больше о нем знаем --> он уже не такой холодный

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install lightfm
!pip install implicit==0.6

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.4/316.4 kB 4.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for lightfm: filename=lightfm-1.17-cp310-cp310-linux_x86_64.whl size=808331 sha256=d7c3534ae4690e5e3e64afd9c25ad8a6d5b6ca42d8ee3aa290171148a1bc3302
  Stored in directory: /root/.cache/pip/wheels/4f/9b/7e/0b256f2168511d8fa4dae4fae0200fdbd729eb424a912ad636
Successfully built lightfm
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.6/18.6 MB 13.3 MB/s eta 0:00:00


In [7]:
import pandas as pd  # Импорт библиотеки pandas под псевдонимом pd
import numpy as np  # Импорт библиотеки numpy под псевдонимом np
import matplotlib.pyplot as plt  # Импорт модуля pyplot из библиотеки matplotlib под псевдонимом plt

# Магическая команда для отображения графиков в Jupyter Notebook
%matplotlib inLine

from scipy.sparse import csr_matrix, coo_matrix  # Импорт разреженных матриц csr_matrix и coo_matrix из библиотеки scipy.sparse

from implicit.als import AlternatingLeastSquares  # Импорт модели AlternatingLeastSquares из библиотеки implicit.als
from implicit.nearest_neighbours import bm25_weight, tfidf_weight  # Импорт функций bm25_weight и tfidf_weight для взвешивания

from lightfm import LightFM  # Импорт модели LightFM из библиотеки lightfm
from lightfm.evaluation import precision_at_k, recall_at_k  # Импорт функций precision_at_k и recall_at_k из lightfm.evaluation

import os, sys  # Импорт модулей os и sys для работы с операционной системой

module_path = os.path.abspath(os.path.join(os.pardir))  # Получение пути к родительской директории текущего местоположения
if module_path not in sys.path:  # Проверка, что путь не находится в списке sys.path
    sys.path.append(module_path)  # Добавление пути в список sys.path

# from src.metrics import precision_at_k, recall_at_k  # Импорт пользовательских функций precision_at_k и recall_at_k из файла src.metrics
# from src.utils import prefilter_items  # Импорт пользовательской функции prefilter_items из файла src.utils

# from src.metrics import precision_at_k, recall_at_k
from content.drive.MyDrive.webinar_5.src.utils import prefilter_items



In [8]:
data = pd.read_csv('/content/drive/MyDrive/webinar_5/retail_train.csv')  # Загрузка данных из файла 'retail_train.csv' в переменную data
item_features = pd.read_csv('/content/drive/MyDrive/webinar_5/product.csv')  # Загрузка данных из файла 'product.csv' в переменную item_features
user_features = pd.read_csv('/content/drive/MyDrive/webinar_5/hh_demographic.csv')  # Загрузка данных из файла 'hh_demographic.csv' в переменную user_features

item_features.columns = [col.lower() for col in item_features.columns]  # Приведение названий столбцов item_features к нижнему регистру
user_features.columns = [col.lower() for col in user_features.columns]  # Приведение названий столбцов user_features к нижнему регистру

item_features.rename(columns={'product_id': 'item_id'}, inplace=True)  # Переименование столбца 'product_id' в 'item_id' в item_features
user_features.rename(columns={'household_key': 'user_id'}, inplace=True)  # Переименование столбца 'household_key' в 'user_id' в user_features

test_size_weeks = 3  # Задание размера тестовой выборки в неделях

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]  # Создание обучающей выборки до последних 3 недель данных
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]  # Создание тестовой выборки из последних 3 недель данных

data_train.head(5)  # Вывод первых 5 строк обучающей выборки data_train


,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.60,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.00,1631,1,0.0,0.0
2,2375,26984851472,1,1036325,1,0.99,364,-0.30,1631,1,0.0,0.0
3,2375,26984851472,1,1082185,1,1.21,364,0.00,1631,1,0.0,0.0
4,2375,26984851472,1,8160430,1,1.50,364,-0.39,1631,1,0.0,0.0


In [9]:
item_features.head()  # Вывод первых нескольких строк (по умолчанию 5 строк) из датафрейма item_features

,item_id,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product
0,25671,2,GROCERY,National,FRZN ICE,ICE - CRUSHED/CUBED,22 LB
1,26081,2,MISC. TRANS.,National,NO COMMODITY DESCRIPTION,NO SUBCOMMODITY DESCRIPTION,
2,26093,69,PASTRY,Private,BREAD,BREAD:ITALIAN/FRENCH,
3,26190,69,GROCERY,Private,FRUIT - SHELF STABLE,APPLE SAUCE,50 OZ
4,26355,69,GROCERY,Private,COOKIES/CONES,SPECIALTY COOKIES,14 OZ


In [10]:

user_features.head()  # Вывод первых нескольких строк (по умолчанию 5 строк) из датафрейма user_features


,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc,user_id
0,65+,A,35-49K,Homeowner,2 Adults No Kids,2,None/Unknown,1
1,45-54,A,50-74K,Homeowner,2 Adults No Kids,2,None/Unknown,7
2,25-34,U,25-34K,Unknown,2 Adults Kids,3,1,8
3,25-34,U,75-99K,Homeowner,2 Adults Kids,4,2,13
4,45-54,B,50-74K,Homeowner,Single Female,1,None/Unknown,16


In [11]:
user_features['age_desc'].unique()  # Извлечение уникальных значений из столбца 'age_desc' в датафрейме user_features

user_features['marital_status_code'].unique()  # Извлечение уникальных значений из столбца 'marital_status_code' в датафрейме user_features

user_features['household_size_desc'].unique()  # Извлечение уникальных значений из столбца 'household_size_desc' в датафрейме user_features


array(['2', '3', '4', '1', '5+'], dtype=object)

## 1. Filter items

In [13]:
n_items_before = data_train['item_id'].nunique()  # Вычисление количества уникальных товаров до фильтрации

data_train = prefilter_items(data_train, item_features)  # Применение функции prefilter_items к обучающему набору данных

n_items_after = data_train['item_id'].nunique()  # Вычисление количества уникальных товаров после фильтрации

print('Decreased # items from {} to {}'.format(n_items_before, n_items_after))  # Вывод информации об уменьшении количества товаров


Decreased # items from 86865 to 5001


## 2. Prepare data set

In [14]:
user_item_matrix = pd.pivot_table(data_train,  # Создание сводной таблицы на основе обучающего набора данных
                                  index='user_id', columns='item_id',  # Определение пользователей в строках и товаров в столбцах
                                  values='quantity',  # Значение - количество товара
                                  aggfunc='count',  # Агрегационная функция - подсчет количества
                                  fill_value=0  # Заполнение пустых значений нулями
                                 )

user_item_matrix = user_item_matrix.astype(float)  # Преобразование матрицы в тип данных float

sparse_user_item = csr_matrix(user_item_matrix).tocsr()  # Преобразование матрицы в разреженный формат CSR (Compressed Sparse Row)

user_item_matrix.head()  # Вывод первых двух строк матрицы пользователь-товар


item_id,818980,818981,819063,819112,819304,819330,819927,820082,820291,820321,...,13512965,13654702,13672065,13777104,13842090,13842107,13842160,13842224,13907063,13910669
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [15]:
# Фильтрация тестового набора данных, оставляя только записи с товарами, присутствующими в обучающем наборе
data_test = data_test[data_test['item_id'].isin(data_train['item_id'].unique())]

# Создание матрицы пользователь-товар для тестового набора данных
test_user_item_matrix = pd.pivot_table(data_test,
                                  index='user_id', columns='item_id',
                                  values='quantity',
                                  aggfunc='count',
                                  fill_value=0
                                 )

# Преобразование значений в матрице в тип данных float
test_user_item_matrix = test_user_item_matrix.astype(float)


In [16]:
# Получение массивов значений идентификаторов пользователей и товаров из матрицы пользователь-товар
userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

# Создание массивов идентификаторов в диапазоне длины массивов userids и itemids
matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

# Создание словаря для отображения идентификаторов товаров на их фактические id
id_to_itemid = dict(zip(matrix_itemids, itemids))

# Создание словаря для отображения идентификаторов пользователей на их фактические id
id_to_userid = dict(zip(matrix_userids, userids))

# Создание словаря для отображения фактических id товаров на их идентификаторы
itemid_to_id = dict(zip(itemids, matrix_itemids))

# Создание словаря для отображения фактических id пользователей на их идентификаторы
userid_to_id = dict(zip(userids, matrix_userids))


## 3. Prepare user and item features

In [17]:
# Создание DataFrame для признаков пользователей на основе их идентификаторов из матрицы пользователь-товар
user_feat = pd.DataFrame(user_item_matrix.index)

# Объединение DataFrame пользовательских признаков с информацией о пользователях по 'user_id'
user_feat = user_feat.merge(user_features, on='user_id', how='left')

# Установка 'user_id' в качестве индекса DataFrame пользовательских признаков
user_feat.set_index('user_id', inplace=True)

# Создание DataFrame для признаков товаров на основе их идентификаторов из матрицы пользователь-товар
item_feat = pd.DataFrame(user_item_matrix.columns)

# Объединение DataFrame признаков товаров с информацией о товарах по 'item_id'
item_feat = item_feat.merge(item_features, on='item_id', how='left')

# Установка 'item_id' в качестве индекса DataFrame признаков товаров
item_feat.set_index('item_id', inplace=True)

user_feat.head()  # Вывод первых строк DataFrame с признаками пользователей


,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc
user_id,,,,,,,
1,65+,A,35-49K,Homeowner,2 Adults No Kids,2,None/Unknown
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
# Преобразование категориальных признаков пользователей в бинарные формы с помощью one-hot кодирования
user_feat_lightfm = pd.get_dummies(user_feat, columns=user_feat.columns.tolist())

# Преобразование категориальных признаков товаров в бинарные формы с помощью one-hot кодирования
item_feat_lightfm = pd.get_dummies(item_feat, columns=item_feat.columns.tolist())


In [19]:
user_feat_lightfm.head(5)

,age_desc_19-24,age_desc_25-34,age_desc_35-44,age_desc_45-54,age_desc_55-64,age_desc_65+,marital_status_code_A,marital_status_code_B,marital_status_code_U,income_desc_100-124K,...,hh_comp_desc_Unknown,household_size_desc_1,household_size_desc_2,household_size_desc_3,household_size_desc_4,household_size_desc_5+,kid_category_desc_1,kid_category_desc_2,kid_category_desc_3+,kid_category_desc_None/Unknown
user_id,,,,,,,,,,,,,,,,,,,,,
1,0,0,0,0,0,1,1,0,0,0,...,0,0,1,0,0,0,0,0,0,1
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [21]:
# ?LightFM

In [22]:
# Создание модели LightFM с указанными параметрами
model = LightFM(no_components=30,
                loss='bpr',  # Функция потерь для обучения модели: Bayesian Personalized Ranking
                learning_rate=0.05,
                item_alpha=0.1, user_alpha=0.1,
                random_state=42)

# Обучение модели на данных
model.fit((sparse_user_item > 0) * 1,  # Разреженная матрица пользователь-товар в бинарном формате (0 и 1)
          sample_weight=coo_matrix(user_item_matrix),  # Веса элементов разреженной матрицы

          user_features=csr_matrix(user_feat_lightfm.values).tocsr(),  # Признаки пользователей в формате CSR
          item_features=csr_matrix(item_feat_lightfm.values).tocsr(),  # Признаки товаров в формате CSR

          epochs=15,  # Количество эпох обучения модели
          num_threads=4)  # Количество потоков для обучения


In [23]:
# Получение векторных представлений пользователей из обученной модели LightFM
user_emb = model.get_user_representations(features=csr_matrix(user_feat_lightfm.values).tocsr())

# Векторы пользователей (user_emb[0]) и их смещения (user_emb[1])
user_vectors = user_emb[0]  # Векторы пользователей
user_bias = user_emb[1]  # Смещения пользователей

# Вывод размерности векторов пользователей
print(user_vectors.shape)  # Размерность векторов пользователей
print(user_bias.shape)     # Размерность смещений пользователей


(2497,)
(2497, 30)


In [24]:
# Получение векторных представлений товаров из обученной модели LightFM
item_emb = model.get_item_representations(features=csr_matrix(item_feat_lightfm.values).tocsr())

# item_emb[0] содержит смещения (bias) для каждого товара
print(item_emb[0].shape)  # Размерность смещений для каждого товара

# item_emb[1] содержит векторные представления (эмбеддинги) товаров
print(item_emb[1].shape)  # Размерность векторных представлений товаров

# Вывод эмбеддингов товаров
item_emb[1]  # Векторные представления (эмбеддинги) товаров


(5001,)
(5001, 30)


array([[-3.6347131e-03, -4.1122236e-03,  5.7189097e-04, ...,
         2.0449189e-03, -3.1994144e-03, -7.9155294e-03],
       [-7.0763903e-04,  5.0500908e-05,  1.0423827e-03, ...,
        -7.9051254e-04, -2.4577179e-03, -1.0167045e-03],
       [-1.0934893e-02, -1.8348653e-03, -1.5752228e-03, ...,
         1.8129797e-03, -2.2834097e-03, -1.6516381e-03],
       ...,
       [-5.2837329e-04, -7.9637958e-04, -1.5583989e-04, ...,
        -6.5285794e-04, -5.6374329e-03, -5.0818273e-03],
       [-4.4773618e-04,  2.4716009e-04, -2.4033997e-02, ...,
         2.3253957e-02,  4.1294406e-04,  3.8714001e-03],
       [ 3.8047903e-03, -1.7340641e-02, -1.4024256e-01, ...,
         7.5339444e-02,  9.5591014e-03, -2.2032838e-02]], dtype=float32)

In [25]:
# Вычисление средней точности (precision) на обучающем наборе данных для модели LightFM
train_precision = precision_at_k(model, sparse_user_item,
                                 user_features=csr_matrix(user_feat_lightfm.values).tocsr(),
                                 item_features=csr_matrix(item_feat_lightfm.values).tocsr(),
                                 k=5).mean()

train_precision  # Вывод средней точности (precision) на обучающем наборе данных


0.22242692

In [26]:
# Заданные идентификаторы товаров для выполнения предсказаний
test_item_ids = np.array([1, 2, 3, 200, 1200])

# Выполнение предсказаний модели LightFM для конкретного пользователя и набора товаров
predictions = model.predict(user_ids=0, item_ids=test_item_ids,
                            user_features=csr_matrix(user_feat_lightfm.values).tocsr(),
                            item_features=csr_matrix(item_feat_lightfm.values).tocsr(),
                            num_threads=4)


In [27]:
predictions

array([-131.62473, -131.7868 , -131.94055, -132.92143, -130.92102],
      dtype=float32)